In [1]:
import keras

Using TensorFlow backend.


In [2]:
import tensorflow as tf
print(tf.__version__)

1.15.0


In [3]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

Instructions for updating:
non-resource variables are not supported in the long term


## Import Libraries 

In [4]:
import pandas as pd
import h5py

In [5]:
import numpy as np
from scipy import interp
import matplotlib.pyplot as plt
from itertools import cycle
from sklearn.metrics import roc_curve, auc
from sklearn.multiclass import OneVsRestClassifier
from sklearn.datasets import make_classification
from sklearn.preprocessing import label_binarize
from scipy import interp
from sklearn.metrics import confusion_matrix
import seaborn as sn
from sklearn.metrics import classification_report

In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import glob
import os
from sklearn.model_selection import train_test_split

%matplotlib inline
plt.rcParams['figure.figsize'] = (5.0, 4.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

In [7]:
import warnings
warnings.simplefilter(action='ignore', category = FutureWarning )

In [8]:
from collections import defaultdict

In [9]:
from keras.preprocessing import image
from keras.applications.imagenet_utils import preprocess_input
from keras.utils import to_categorical

from keras import layers
from keras.layers import Input, Dense, Activation, Dropout, BatchNormalization, Flatten
from keras.layers import AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D, GlobalAveragePooling2D , Flatten, Conv2D, ZeroPadding2D
from keras.models import Sequential

from keras.optimizers import RMSprop , SGD, Adam
from keras.callbacks import ModelCheckpoint

import keras.backend as K
from keras.models import load_model

K.set_image_data_format('channels_last')




In [10]:

from keras.applications.resnet_v2 import ResNet50V2

In [11]:
from keras.models import Model
from keras.applications.inception_v3 import InceptionV3
from keras.layers import Input

In [12]:
%load_ext autoreload
%autoreload 2

In [13]:
from PIL import Image
import cv2

### Metric Function

In [14]:
def metrics(model, X_test, Y_test):
    snn_pred = model.predict(X_test, batch_size=32, verbose=1) 
    snn_predicted = np.argmax(snn_pred, axis=1)
    
    model_metrics= model.evaluate(X_test, Y_test)
    print(model)
    print("model accuracy on test data is : {}" .format(model_metrics[1]))
    #Confusion Matrix
    snn_cm = confusion_matrix(np.argmax(Y_test, axis=1), snn_predicted) 

    # Visualization
    snn_df_cm = pd.DataFrame(snn_cm, range(14), range(14)) 
    plt.figure(figsize = (20,14)) 
    sn.set(font_scale=1.4) #for label size 
    sn.heatmap(snn_df_cm, annot=True, annot_kws={"size": 12}) # font size 
    plt.show()
    
    snn_report = classification_report(np.argmax(Y_test, axis=1), snn_predicted)
    print(snn_report)
    
    # Plot linewidth.
    lw = 2

    # Compute ROC curve and ROC area for each class
    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    for i in range(num_classes):
        fpr[i], tpr[i], _ = roc_curve(Y_test[:, i], snn_pred[:, i])
        roc_auc[i] = auc(fpr[i], tpr[i])

    # Compute micro-average ROC curve and ROC area
    fpr["micro"], tpr["micro"], _ = roc_curve(Y_test.ravel(), snn_pred.ravel())
    roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

    # Compute macro-average ROC curve and ROC area

    # First aggregate all false positive rates
    all_fpr = np.unique(np.concatenate([fpr[i] for i in range(num_classes)]))

    # Then interpolate all ROC curves at this points
    mean_tpr = np.zeros_like(all_fpr)
    for i in range(num_classes):
        mean_tpr += interp(all_fpr, fpr[i], tpr[i])

    # Finally average it and compute AUC
    mean_tpr /= num_classes

    fpr["macro"] = all_fpr
    tpr["macro"] = mean_tpr
    roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])

    # Plot all ROC curves
    plt.figure(1, figsize=(15, 10), dpi=80)
   
    colors = cycle(['aqua', 'darkorange', 'cornflowerblue', 'pink', 'black', 'red', 'yellow', 'green', 'brown', 'black', 'purple','indigo','darkgreen','darkblue'])
    for i, color in zip(range(num_classes), colors):
        plt.plot(fpr[i], tpr[i], color=color, lw=lw,
                label='ROC curve of class {0} (area = {1:0.2f})'
                ''.format(i, roc_auc[i])) 

    plt.plot([0, 1], [0, 1], 'k--', lw=lw)
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver operating characteristic to multi-class')
    plt.legend(loc="lower right")
    plt.show()

### Import data

In [15]:
# from google.colab import drive

# drive.mount('/content/gdrive')
# root_path = 'gdrive/My Drive/SML project/'

In [16]:
import h5py
from pathlib import Path
root_path = ''
hdf5_dir = Path(root_path + "data/hdf5/")
print(hdf5_dir)

data/hdf5


In [17]:
def read_hdf5(path):
    
    train_images, train_labels = [], []
    test_images, test_labels = [], []

    # Open the HDF5 file
    file = h5py.File(hdf5_dir / "train_test_data.h5", "r+")

    train_images = np.array(file["/train_images"]).astype("uint8")
    train_labels = np.array(file["/train_meta"]).astype("uint8")
    test_images = np.array(file["/test_images"]).astype("uint8")
    test_labels = np.array(file["/test_meta"]).astype("uint8")

    return train_images, train_labels ,test_images, test_labels


In [18]:
X_train_input , Y_train_input, X_test_input, Y_test_input = read_hdf5('train_test_data')

In [19]:
print(X_train_input.shape)
print(X_test_input.shape)
print(Y_train_input.shape)
print(Y_test_input.shape)

(18966, 256, 256, 3)
(4742, 256, 256, 3)
(18966, 3)
(4742, 3)


In [20]:
num_classes = int(np.max(Y_train_input[:, 2]))
print("number of classes are: {}" .format(num_classes))
#dummy_classes_var = len(np.unique(Y_train_input[:, 2]))
#print( " dummy classes variable test {}" .format(dummy_classes_var))

number of classes are: 14


### Initial preprocess

In [ ]:
X_train_input1 = X_train_input.astype('float32')
X_test_input1 = X_test_input.astype('float32')

In [ ]:
X_train_input1 = X_train_input1/255.0
X_test_input1 = X_test_input1/255.0 

In [ ]:
X_train_preprocessed = preprocess_input(X_train_input)
print(X_train_preprocessed.shape)
X_test_preprocessed = preprocess_input(X_test_input)
print(X_test_preprocessed.shape)

In [ ]:
### With preprocess
plt.imshow(X_train_preprocessed[100])

In [ ]:
### without preprocess
plt.imshow(X_train_input1[100])

In [ ]:
Y_train_age_group = Y_train_input[:, 2]
print(Y_train_age_group)
Y_train_age_group_0 = np.subtract(Y_train_age_group , 1)
print(Y_train_age_group_0)
Y_train_age_group_onehot = to_categorical(Y_train_age_group_0, num_classes= num_classes)
print(Y_train_age_group_onehot.shape)

In [ ]:
Y_test_age_group = Y_test_input[:, 2]
print(Y_test_age_group)
Y_test_age_group_0 = np.subtract(Y_test_age_group , 1)
print(Y_test_age_group_0)
Y_test_age_group_onehot = to_categorical(Y_test_age_group_0, num_classes= num_classes)
print(Y_test_age_group_onehot.shape)

In [ ]:


### Assign train and test data
X_train = X_train_preprocessed

X_test = X_test_preprocessed

Y_train = Y_train_age_group_onehot

Y_test = Y_test_age_group_onehot

pixel_size = int(X_train.shape[1])

#num_examples = int(X_train_original.shape[0])

print("number of classes are : {}" .format(num_classes))

input_shape = (pixel_size, pixel_size , 3)



In [ ]:
sum(Y_train[: , 4] == 1)

### Data Augmentation

In [22]:
from keras.preprocessing.image import ImageDataGenerator

In [23]:
# This will do preprocessing and realtime data augmentation:
datagen = ImageDataGenerator(
            # set input mean to 0 over the dataset
            featurewise_center=True,              ### changed to true
            # set each sample mean to 0
            samplewise_center=False,
            # divide inputs by std of dataset
            featurewise_std_normalization=True,        ### changed to true
            # divide each input by its std
            samplewise_std_normalization=False,
            # apply ZCA whitening
            zca_whitening=False,
            # epsilon for ZCA whitening
            zca_epsilon=1e-06,
            # randomly rotate images in the range (deg 0 to 180)
            rotation_range=10,
            # randomly shift images horizontally
            width_shift_range=0.1,
            # randomly shift images vertically
            height_shift_range=0.1,
            # set range for random shear
            shear_range=0.2,
            # set range for random zoom
            zoom_range=0.2,
            # set range for random channel shifts
            channel_shift_range=0.,
            # set mode for filling points outside the input boundaries
            fill_mode='nearest',
            # value used for fill_mode = "constant"
            cval=0.,
            # randomly flip images
            horizontal_flip=True,
            # randomly flip images
            vertical_flip=False,
            # set rescaling factor (applied before any other transformation)
            rescale=None,
            # set function that will be applied on each input
            preprocessing_function=None,
            # image data format, either "channels_first" or "channels_last"
            data_format='channels_last',
            # fraction of images reserved for validation (strictly between 0 and 1)
            validation_split=0.2)

        # Compute quantities required for featurewise normalization
        # (std, mean, and principal components if ZCA whitening is applied).


In [26]:
#datagen.fit(X_train)

In [25]:
# Run training, with or without data augmentation.

def train_func(model ,x_train, y_train, datagen , data_augmentation = False, batch_size = 64, epochs = 10, validation_split =0.2 , \
               callbacks = None,\
               root_path='' , save_model = True, model_name=''  ):
    if not data_augmentation:
        print('Not using data augmentation.')
        model_history = model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_split=validation_split,
              shuffle=True,
              callbacks=callbacks)
        data_aug = 'noaug'
    else:
        print('Using real-time data augmentation.')
        
        print("fitting the model using fit_generator !!!!")
        # Fit the model on the batches generated by datagen.flow().
        model_history = model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),
                        epochs=epochs, shuffle=True,workers = 4, 
                        callbacks=callbacks)
        data_aug = 'aug'
    if save_model:
        model.save(root_path + 'saved_models/' + "{}_{}_{}_{}.h5".format(model_name,data_aug, epochs, batch_size ))    
    return model_history


**Callbacks preparation:**



In [44]:
# Prepare model model saving directory.
version = 1
model_type = 'Resnet50V2%d' % (version)
save_dir = os.path.join(root_path, 'saved_models')
print(save_dir)
model_name = 'Resnet50V2_%s_model.{epoch:03d}.h5' % model_type
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
filepath = os.path.join(save_dir, model_name)

# Prepare callbacks for model saving and for learning rate adjustment.
resnet_model_pretrain_checkpoint = ModelCheckpoint(filepath=filepath,
                             monitor='val_accuracy',
                             verbose=1,
                             save_best_only=True)
resnet_model_pretrain_callbacks = [resnet_model_pretrain_checkpoint]

saved_models


# Building Models 

In [36]:
adam = Adam(lr=0.01, beta_1=0.9, beta_2=0.99, amsgrad=False)
sgd = SGD(lr=0.01, momentum=0.9, nesterov=True)
rms_prop = RMSprop(lr=0.001, rho =0.9)

### Resnet 50 v2

In [37]:
input_tensor = Input(shape=(256, 256, 3)) 
original_model = ResNet50V2(include_top=True, weights='imagenet', 
                               input_tensor=input_tensor, pooling='Max', classes=1000)


In [38]:
original_model.summary()

Model: "resnet50v2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 262, 262, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 128, 128, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
pool1_pad (ZeroPadding2D)       (None, 130, 130, 64) 0           conv1_conv[0][0]                 
_________________________________________________________________________________________

In [39]:

bottleneck_input  = original_model.get_layer(index=0).input
bottleneck_output = original_model.get_layer(index=-2).output
bottleneck_model  = Model(inputs=bottleneck_input, outputs=bottleneck_output)

In [40]:
bottleneck_model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 262, 262, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 128, 128, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
pool1_pad (ZeroPadding2D)       (None, 130, 130, 64) 0           conv1_conv[0][0]                 
____________________________________________________________________________________________

In [41]:
resnet_model_pretrain = Sequential()
resnet_model_pretrain.add(bottleneck_model)
resnet_model_pretrain.add(Dense(512, activation='relu', input_dim=2048))
resnet_model_pretrain.add(Dense(num_classes, activation='softmax', input_dim=512))

In [42]:
resnet_model_pretrain.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
model_1 (Model)              (None, 2048)              23564800  
_________________________________________________________________
dense_1 (Dense)              (None, 512)               1049088   
_________________________________________________________________
dense_2 (Dense)              (None, 14)                7182      
Total params: 24,621,070
Trainable params: 24,575,630
Non-trainable params: 45,440
_________________________________________________________________


In [43]:
resnet_model_pretrain.compile(loss= 'categorical_crossentropy' ,
              optimizer = sgd,
               metrics=['accuracy'] )

In [46]:
resnet_model_pretrain_history = train_func(model = resnet_model_pretrain ,x_train = X_train_input1, y_train = Y_train, \
             datagen = datagen ,data_augmentation = False, batch_size = 64, callbacks = resnet_model_pretrain_callbacks , \
           epochs = 40, validation_split =0.2 , \
               root_path = '', save_model = True, model_name= 'resnet50V2')

Not using data augmentation.
Train on 15172 samples, validate on 3794 samples
Epoch 1/40
15172/15172 [==============================] - 127s 8ms/step - loss: 0.2227 - accuracy: 0.9260 - val_loss: 2.7924 - val_accuracy: 0.3727

Epoch 00001: val_accuracy did not improve from 0.39141
Epoch 2/40
15172/15172 [==============================] - 127s 8ms/step - loss: 0.1927 - accuracy: 0.9378 - val_loss: 3.2142 - val_accuracy: 0.3743

Epoch 00002: val_accuracy did not improve from 0.39141
Epoch 3/40
15172/15172 [==============================] - 127s 8ms/step - loss: 0.1996 - accuracy: 0.9382 - val_loss: 3.4563 - val_accuracy: 0.3490

Epoch 00003: val_accuracy did not improve from 0.39141
Epoch 4/40
15172/15172 [==============================] - 127s 8ms/step - loss: 0.1687 - accuracy: 0.9465 - val_loss: 3.1875 - val_accuracy: 0.3806

Epoch 00004: val_accuracy did not improve from 0.39141
Epoch 5/40
15172/15172 [==============================] - 127s 8ms/step - loss: 0.1307 - accuracy: 0.9586 

In [ ]:
plt.figure()
plt.plot(resnet_model_pretrain_history.history['loss'])
plt.plot(resnet_model_pretrain_history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

In [ ]:

# Plot training & validation accuracy values
plt.plot(resnet_model_pretrain_history.history['accuracy'])
plt.plot(resnet_model_pretrain_history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

In [ ]:
metrics(model = resnet_model_pretrain, X_test = X_test_input1 , Y_test = Y_test)

In [ ]:
plot_model(resnet_model_pretrain , to_file = 'resnet50V2_model.png')


In [1]:
SVG(model_to_dot(resnet_model_pretrain).create(prog='dot', format = 'svg'))

NameError: name 'SVG' is not defined